## Fera Formidável 4.1
### Quem classifica a classe classificadora?
#### Rômulo 24024

### Enunciado:
**Objetivo:** altere a rede neural feita em Python puro para resolver um problema de
classificação. Treine uma rede neural em um dataset simples de classificação para mostrar
que funciona.

**Comentário:** aqui é necessário se informar sobre as diferenças de uma rede neural
classificadora com relação a uma rede neural regressora. A função de perda, por exemplo,
não poderá ser mais a função de perda dos resíduos quadrados.

**Comentário 2:** observe que o enunciado diz claramente que é para realizar a tarefa
na rede neural feita em Python puro nos vídeos da disciplina. Se você está usando o
PyTorch, numpy, tensorflow, keras, lightning ou qualquer outra biblioteca pronta,
você está no caminho errado!

### Introdução:
Resolverei um problema de classificação binária.
Identifiquei as seguintes alterações necessárias para fazer para transformar um problema de classificação binária:
- A função de ativação sigmoidal parece funcionar bem para classificação binária, será retornada ao final da rede neural a previsão de probabilidade de um dos rótulos.
- A função de erro não pode ser mais RMSE, escolhi para substituí-a a função de perda logarítma(log loss), que é uma aplicação da função de perda de entropia cruzada(cross entropy)

#### Imports necessários:

In [14]:
import pandas as pd
import random as random
import math
import numpy as np

#### Classes da Rede Neural  em python puro:
- Foi adicionada a operação logarítma na classe 'Valor' para realização da função de perda logarítma

In [15]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = np.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    def log(self):
        """Calcula o logaritmo natural do número"""
        progenitor = (self,)
        data = math.log(self.data)
        resultado = Valor(data, progenitor, "log")

        def propagar_log():
            self.grad += resultado.grad * (1 / self.data)

        resultado.propagar = propagar_log
        return resultado

    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    def __rsub__(self, outro_valor):
        """Realiza a operação: outro_valor - self"""
        return outro_valor + (-self)
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [16]:
class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        dado_de_saida = soma.sig()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

In [17]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [18]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

### Dados:
O dataset utilizado contém dados sobre diagnóstico de tumores malignos ou benignos localizado na mama, contém 569 amostras e 30 atributos numéricos. Os fatores levados em consideração serão:

a) Raio 

b) Textura

c) Perímetro

d) Área

e) Suavidade 

f) Compacidade 

g) Concavidade 

h) Pontos côncavos 

i) Simetria 

j) Dimensão fractal 

Cada um desses fatores tem uma média, um desvio padrão e um valor máximo como atributos.das células. Esses atributos são derivados de imagens digitalizadas de biópsias de mama. O target será a avaliação se um tumor é benigno (0) ou maligno (1), que foi obtido no dataset com base em medidas obtidas por exames de imagem. 

In [19]:
nomes_colunas = [
    'ID', 'diagnóstico',
    'raio_m', 'textura_m', 'perímetro_m', 'área_m', 'suavidade_m', 'compacidade_m', 'concavidade_m', 'pontos_côncavos_m', 'simetria_m', 'dimensao_fractal_m',
    'raio_sd', 'textura_sd', 'perímetro_sd', 'área_sd', 'suavidade_sd', 'compacidade_sd', 'concavidade_sd', 'pontos_côncavos_sd', 'simetria_sd', 'dimensao_fractal_sd',
    'raio_max', 'textura_max', 'perímetro_max', 'área_max', 'suavidade_max', 'compacidade_max', 'concavidade_max', 'pontos_côncavos_max', 'simetria_max', 'dimensao_fractal_max'
]

dados = pd.read_csv("C:\\Users\\romulo24024\\OneDrive - ILUM ESCOLA DE CIÊNCIA\\RedesNeuraisAlgoritmosGenéticos\\Datasets\\bcancer\\wdbc.data", header=None, names=nomes_colunas)
dados['diagnóstico'] = dados['diagnóstico'].apply(lambda x: 1 if x == 'M' else 0)

In [20]:
X = dados.drop(['diagnóstico', 'ID'], axis=1)
y_true = dados['diagnóstico']

In [21]:
X

,raio_m,textura_m,perímetro_m,área_m,suavidade_m,compacidade_m,concavidade_m,pontos_côncavos_m,simetria_m,dimensao_fractal_m,...,raio_max,textura_max,perímetro_max,área_max,suavidade_max,compacidade_max,concavidade_max,pontos_côncavos_max,simetria_max,dimensao_fractal_max
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [22]:
x = [ list(X.iloc[i]) for i in range(len(X))]

#### Treinando a MLP:
Como teremos 30 dados de entrada, devemos ajustar o nosso número de dados de entrada.

In [23]:
NUM_DADOS_DE_ENTRADA = 30 
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3,2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

minha_mlp = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

Para obtermos uma função de perda corretamente, calculando com base na probabilidade de previsão, devemos aplicar a função de perda logaritma.

In [24]:
NUM_EPOCAS = 10
TAXA_DE_APRENDIZADO = 0.1

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = minha_mlp(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        log_loss = Valor(yt) * yp.log() + (1 - Valor(yt)) * (1-yp).log()
        erros.append(log_loss)      
    loss = -sum(erros)/ len(erros)

    # zero grad
    for p in minha_mlp.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

C:\Users\romulo24024\AppData\Local\Temp\ipykernel_28520\1155448677.py:53: RuntimeWarning: overflow encountered in exp
  data = np.exp(self.data)
C:\Users\romulo24024\AppData\Local\Temp\ipykernel_28520\1155448677.py:38: RuntimeWarning: invalid value encountered in scalar multiply
  data = self.data * outro_valor.data


0 nan
1 nan
2 nan
3 nan
4 nan


KeyboardInterrupt: 

In [ ]:
normalizar dados de entrada:
sugestão maximo absoluto